In [16]:
#Add libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
%matplotlib inline

In [2]:
#Load data processed on the HPC

prs = pd.read_csv("../data/protein_prs_cases.csv", sep="\t", index_col=0)

prs.head()

,Group,Case,Sex,C4A.C4B.4481.34.2,IL10.2773.50.2,MMP9.2579.17.5,CSF3.8952.65.3,GC.6581.50.3,APOB.2797.56.2,CFH.4159.130.1,...,C3.2755.8.2,PPY.4588.1.2,IGFBP2.2570.72.5,APOE.2937.10.2,FGA.FGB.FGG.4907.56.1,PLG.3710.49.2,TNF.5936.53.3,ANGPT2.13660.76.3,CRP.4337.49.2,VCAM1.2967.8.1
0,A,2,2,-0.069435,-0.034924,-0.126787,0.351996,0.314848,-0.318171,0.195647,...,0.057947,0.749855,0.146249,-0.133883,-2.490502,-0.324963,-0.978661,0.039893,-0.266216,-0.038887
1,A,2,2,-1.071661,-0.421213,-0.872182,0.101832,-0.296282,0.689710,-0.409555,...,-0.903393,0.306309,-1.152134,0.421410,0.405951,0.950770,1.549364,1.410669,-1.659654,2.214628
2,A,2,1,0.321382,-2.000421,-0.126787,0.207792,0.952631,-2.252077,-1.264962,...,1.754089,1.248028,1.187176,-1.003851,-0.848056,-0.324963,-1.076009,0.381713,-0.420168,-0.413070
3,A,2,2,0.575056,-0.256073,0.618608,-1.863235,-0.379302,0.689710,-0.636036,...,-1.259195,-0.329593,-0.662532,-0.133883,1.754016,0.614542,-1.708552,-1.134225,-1.053209,-0.110263
4,A,2,2,1.512781,0.417969,-0.126787,0.091216,-0.337681,-0.318171,-0.115083,...,0.554612,-0.582945,0.512406,0.843965,0.379672,-1.239034,0.858960,-0.683777,-0.344878,-0.487496


In [5]:
#Update case and sex from 2/1 and to dummy variables 1/0
#Case -> 1 = "AD", 0 = "CTL"
#Sex -> 1 = "Female", 0 = "Male"

if (prs["Case"].max() == 2) | (prs["Sex"].max() == 2):
    prs.loc[prs["Case"] == 1, "Case"] = 0
    prs.loc[prs["Case"] == 2, "Case"] = 1

    prs.loc[prs["Sex"] == 1, "Sex"] = 0
    prs.loc[prs["Sex"] == 2, "Sex"] = 1
else:
    print("Already updated")

print(prs[["Case", "Sex"]].describe())

print(prs.groupby("Group")["Case"].sum())

              Case          Sex
count  6244.000000  6244.000000
mean      0.686579     0.598334
std       0.463921     0.490274
min       0.000000     0.000000
25%       0.000000     0.000000
50%       1.000000     1.000000
75%       1.000000     1.000000
max       1.000000     1.000000
Group
A    3277
B     639
C     371
Name: Case, dtype: int64


In [ ]:
#GOAL - predict AD status from 31 proteins using a simple feed forward neural network

#Questions
#-> how represent the input proteins e.g. float vector of 31 values? What dimensions?
#-> how represent the output of AD disease status e.g. a 2 node output layer connected by a softmax / sigmoid function? 
#-> how structure hidden layers? How many?
#-> how initialise layers?
#-> what objective, loss function?
#-> how setup batch sizes for training? e.g. all samples in one batch, randomised sub-samples, 1 sample at a time?
    #-> are batches all calculated at once? e.g. [10, 31] input if batch of 10
#-> when do you need to explicitly set a model to train? (varied across examples, not in RNN, is in CNN images)

#Resources
#-> binary classifier gist - https://gist.github.com/santi-pdp/d0e9002afe74db04aa5bbff6d076e8fe
#-> binary cross entropy loss - https://towardsdatascience.com/understanding-binary-cross-entropy-log-loss-a-visual-explanation-a3ac6025181a 
#-> practical stack overflow example - https://stackoverflow.com/questions/62413462/why-is-a-simple-binary-classification-failing-in-a-feedforward-neural-network
#-> microsoft binary classifier example (+ code) - https://visualstudiomagazine.com/articles/2020/10/14/pytorch-define-network.aspx
#-> more detailed analytics vidya example - https://www.analyticsvidhya.com/blog/2019/01/guide-pytorch-neural-networks-case-studies/
#-> MNIST examples - https://adventuresinmachinelearning.com/pytorch-tutorial-deep-learning/ 
    #->+ https://www.kdnuggets.com/2018/02/simple-starter-guide-build-neural-network.html
    #->+ https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/01-basics/feedforward_neural_network/main.py
    #->+ Highly detailed and some useful code snippets - https://www.deeplearningwizard.com/deep_learning/practical_pytorch/pytorch_feedforward_neuralnetwork/
#-> Titanic example (using Softmax) - https://www.kaggle.com/tauseef6462/simple-feedforward-neural-network-using-pytorch 
#-> other shorter examples - https://medium.com/biaslyai/pytorch-introduction-to-neural-network-feedforward-neural-network-model-e7231cff47cb
    #->+ https://medium.com/@niranjankumarc/building-a-feedforward-neural-network-using-pytorch-nn-module-52b1d7ea5c3e
#-> dataloaders - https://towardsdatascience.com/pytorch-basics-intro-to-dataloaders-and-loss-functions-868e86450047 

In [32]:
#get data into train and test splits and convert to numpy
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(prs.iloc[:, 3:len(prs)].to_numpy(), prs["Case"].to_numpy(), test_size=0.25, random_state=0)

print(x_train.shape, x_train[0:5])
print(y_train.shape, y_train[0:5])

(4683, 31) [[ 1.79822108e+00  1.38564230e+00 -1.26787020e-01 -2.64358581e-01
  -6.64585064e-01 -3.18171165e-01  2.97652982e+00 -9.44792401e-01
   2.07457949e+00 -1.55962957e-02  1.95613404e+00 -7.33284252e-01
  -2.55165589e-01 -2.40230399e-01  2.36572896e+00 -2.52146777e+00
   1.27402325e-01 -3.10296251e-02  1.29081691e-01  2.16085459e-01
  -4.65851479e-01 -9.51843362e-01  8.36060371e-01  5.82307505e-01
   4.21409810e-01 -1.03425498e+00 -2.08494454e+00  2.09337045e+00
  -7.45183857e-01  9.43303409e-01 -4.16962328e-01]
 [-1.19283157e-01 -8.66875549e-01 -1.26787020e-01 -4.39616866e-01
   6.87377628e-02 -1.24419598e+00  1.55258980e+00 -3.12740071e-03
  -4.75072688e-01 -1.22155483e+00 -1.27723900e-02  1.47911251e+00
   1.07026093e-01 -3.31017085e-01  5.16887532e-01  1.81145068e-01
  -8.52389837e-01  5.80389740e-01  5.91351783e-01  6.54761946e-01
   1.52169956e+00  1.21552152e+00  9.06974619e-01  8.17190512e-01
  -3.38466638e+00  6.55822101e-01 -1.71565351e+00 -1.98551263e+00
  -3.60881449e

In [33]:
#convert numpy array to torch tensors
x_train, y_train, x_test, y_test = map(torch.tensor, (x_train, y_train, x_test, y_test))

In [34]:
print(x_train.size(), x_train[0:5])
print(y_train.size(), y_train[0:5])

torch.Size([4683, 31]) tensor([[ 1.7982e+00,  1.3856e+00, -1.2679e-01, -2.6436e-01, -6.6459e-01,
         -3.1817e-01,  2.9765e+00, -9.4479e-01,  2.0746e+00, -1.5596e-02,
          1.9561e+00, -7.3328e-01, -2.5517e-01, -2.4023e-01,  2.3657e+00,
         -2.5215e+00,  1.2740e-01, -3.1030e-02,  1.2908e-01,  2.1609e-01,
         -4.6585e-01, -9.5184e-01,  8.3606e-01,  5.8231e-01,  4.2141e-01,
         -1.0343e+00, -2.0849e+00,  2.0934e+00, -7.4518e-01,  9.4330e-01,
         -4.1696e-01],
        [-1.1928e-01, -8.6688e-01, -1.2679e-01, -4.3962e-01,  6.8738e-02,
         -1.2442e+00,  1.5526e+00, -3.1274e-03, -4.7507e-01, -1.2216e+00,
         -1.2772e-02,  1.4791e+00,  1.0703e-01, -3.3102e-01,  5.1689e-01,
          1.8115e-01, -8.5239e-01,  5.8039e-01,  5.9135e-01,  6.5476e-01,
          1.5217e+00,  1.2155e+00,  9.0697e-01,  8.1719e-01, -3.3847e+00,
          6.5582e-01, -1.7157e+00, -1.9855e+00, -3.6088e-02, -1.0532e+00,
          1.5110e-01],
        [ 1.1733e+00,  3.5215e-01, -1.2679e

In [35]:
#setup torch datasets to load into dataloaders

class CustomDataset(torch.utils.data.Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    
training_data = CustomDataset(x_train, y_train)
test_data = CustomDataset(x_test, y_test)

In [58]:
#setup dataloaders to handle batches

batch_size = 100
n_iters = 10000
num_epochs = n_iters / (len(x_train) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=training_data, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_data, 
                                           batch_size=batch_size, 
                                           shuffle=False)

In [64]:
#Create a 1-hidden layer neural net (31 -> 50 -> 1) with a SGD optimiser and a BCE Loss objective function

class Net(nn.Module):
    #initializing layers as per Microsoft article (but does not seem to be a requirement)
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(31, 50)
        self.relu1 = nn.ReLU()
        self.out = nn.Linear(50, 1)
        self.out_act = nn.Sigmoid()
        
        torch.nn.init.xavier_uniform_(self.fc1.weight) 
        torch.nn.init.zeros_(self.fc1.bias)
        torch.nn.init.xavier_uniform_(self.out.weight) 
        torch.nn.init.zeros_(self.out.bias)
        
    def forward(self, input):
        a1 = self.fc1(input)
        h1 = self.relu1(a1)
        a2 = self.out(h1)
        y = self.out_act(a2)
        return y
    
net = Net()
opt = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [65]:
#Training

#turn training on 
net.train()

#loop through epochs
for e in range(1,num_epochs+1):
    epoch_loss = 0
    #loop through batches
    for batch_index, batch in enumerate(train_loader):
        X_batch = batch[0].float()
        y_batch = batch[1].float()
        #zero the gradient
        opt.zero_grad()
        
        #predict the output
        y_batch_pred = net(X_batch)
        
        #calculate the loss (check expected function dimensions)
        loss = criterion(y_batch_pred, y_batch.unsqueeze(1))
        
        #compute the gradient
        loss.backward()
        
        #update the weights
        opt.step()
        
        #accumulate epoch losses
        epoch_loss += loss.item()
        
    #present epoch outputs
    print("Epoch: " + str(e) + " | " + "Loss: " + str(epoch_loss))

#Next steps 
    #-> build evaluation to test accuracy of predictions
    #-> review if setup correct / how to add improvements


Epoch: 1 | Loss: 32.719083070755005
Epoch: 2 | Loss: 31.179018139839172
Epoch: 3 | Loss: 30.470041632652283
Epoch: 4 | Loss: 29.989330172538757
Epoch: 5 | Loss: 29.652220726013184
Epoch: 6 | Loss: 29.362329065799713
Epoch: 7 | Loss: 29.154961466789246
Epoch: 8 | Loss: 28.94689705967903
Epoch: 9 | Loss: 28.805283427238464
Epoch: 10 | Loss: 28.654724836349487
Epoch: 11 | Loss: 28.54733121395111
Epoch: 12 | Loss: 28.458424150943756
Epoch: 13 | Loss: 28.364082098007202
Epoch: 14 | Loss: 28.287536025047302
Epoch: 15 | Loss: 28.21243464946747
Epoch: 16 | Loss: 28.157539129257202
Epoch: 17 | Loss: 28.105873227119446
Epoch: 18 | Loss: 28.06497198343277
Epoch: 19 | Loss: 28.00512307882309
Epoch: 20 | Loss: 27.96257209777832
Epoch: 21 | Loss: 27.915198743343353
Epoch: 22 | Loss: 27.87716841697693
Epoch: 23 | Loss: 27.854857087135315
Epoch: 24 | Loss: 27.81659686565399
Epoch: 25 | Loss: 27.789813995361328
Epoch: 26 | Loss: 27.750869691371918
Epoch: 27 | Loss: 27.723398327827454
Epoch: 28 | Loss: 

In [70]:
print(y_batch_pred.view(-1))
print(y_batch)

tensor([0.5751, 0.5911, 0.7144, 0.6459, 0.6608, 0.6427, 0.5824, 0.9222, 0.6920,
        0.5816, 0.7630, 0.3597, 0.7692, 0.8987, 0.4243, 0.6986, 0.4913, 0.6223,
        0.7749, 0.4378, 0.7197, 0.5926, 0.5797, 0.3618, 0.6172, 0.9625, 0.7680,
        0.6439, 0.5291, 0.3537, 0.2973, 0.8067, 0.5505, 0.7727, 0.6406, 0.6414,
        0.6594, 0.2884, 0.6111, 0.3640, 0.9343, 0.6750, 0.6052, 0.5951, 0.6089,
        0.6143, 0.8803, 0.8883, 0.8484, 0.3554, 0.8485, 0.5950, 0.9541, 0.6667,
        0.6288, 0.4770, 0.7058, 0.7369, 0.2867, 0.8017, 0.5314, 0.5680, 0.9002,
        0.4736, 0.6045, 0.8249, 0.6063, 0.8039, 0.5714, 0.7870, 0.8556, 0.6451,
        0.4531, 0.9079, 0.5328, 0.6302, 0.6715, 0.6787, 0.7251, 0.7702, 0.9409,
        0.6854, 0.5490], grad_fn=<ViewBackward>)
tensor([0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1.,
        1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
        1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0.,

In [9]:
#REVIEW IF REQUIRED
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix

def assess_model_performance(md, pred, X_test, y_test, kernel, plot=True):
    accuracy_sk = md.score(X_test, y_test)
    auc = roc_auc_score(y_test, md.predict(X_test))
    tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
    sensitivity = tp / (tp + fn) #-> also recall
    specificity = tn / (tn + fp)
    precision = tp / (tp + fp)
    
    result = pd.DataFrame({"kernel":kernel, 
                           "accuracy":accuracy_sk, 
                           "sensitivity": sensitivity, 
                           "specificity":specificity,
                           "precision":precision,
                           "auc":auc}, index=[1])
    
    if plot:
        probas = md.predict_proba(X_test)
        plt.plot(roc_curve(y_test, probas[:,1])[0], roc_curve(y_test, probas[:,1])[1], label=kernel)
        plt.xlabel("FPR")
        plt.ylabel("TPR")
        plt.legend(prop={'size':10}, loc='lower right')
    
    return result